**Introduction**

In online multiplayer games, not all players engage at the same level. Some log in for quick sessions, while others contribute much more by playing consistently and for longer periods. Game developers and researchers want to understand what factors drive player engagement so they can design better experiences and boost engagement. In particular, characteristics like a player's age and game experience might influence how actively they participate. 

In this project, we explore data collected from Plaicraft, a Minecraft research server created by the PLAI research group in the UBC Department of Computer Science, led by Professor Frank Wood. The objective of running Plaicraft is to obtain human gameplay data to train artificial general intelligence (AGI) agents for video games (Smith, 2025). The provided dataset includes logs of player sessions and basic player information, allowing us to analyze how individual characteristics relate to gaming behaviours.


Our research question is: Can a player’s game experience and age help predict if they’ll be a high contributor (i.e., someone who plays more than half an hour)? Identifying high contributors is important because they will generate richer gameplay data that researchers can use to train AGI models.

Dataset Description:

The given dataset is made up of two structured files:

*players.csv* - Contains information of 197 unique players and includes the 7 following variables:
- experience: categorical, self-reported skill level (Pro, Veteran, Regular, Amateur)
- subscribe: Boolean, whether the player subscribed to the server newsletter
- hashedEmail: identifier, anonymized unique player ID
- played_hours: numeric, total hours the player has spent in-game
- name: string, player's chosen username
- gender: categorical, player's gender identity (Male, Female, Non-binary etc)
- Age: numeric, player's age in years 


*sessions.csv* - Contains 1536 individual gameplay sessions with the 5 following variables:
- hashedEmail: identifier, anonymized unique player ID
- start_time and end_time: start and end of each gameplay session
- original_start_time and original_end_time: unix epoch timestaps


Player characteristics (e.g., age, experience, and gender) were self-reported, while gameplay sessions were recorded automatically through screen monitoring. Consequently, there may be some subjectivity or inaccuracy in self-reported variables, especially in categorical fields like experience. However, since the game server automatically disconnected players who were inactive for more than 3 minutes, we can be confident that the recorded playtime is reliable.

For this project, we focus on the players.csv file, since it contains the variables needed to evaluate how player characteristics relate to total playtime, which we use to determine if a player is a high contributor.


player csv link https://raw.githubusercontent.com/ashleyan1207/DSCI-100-Project/refs/heads/main/players.csv

sessions csv link https://raw.githubusercontent.com/ashleyan1207/DSCI-100-Project/refs/heads/main/sessions.csv